In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.optimize as sopt
import pandas as pd
import os
from pyomo.environ import *
from idaes.core.solvers import get_solver
import pandas as pd

In [2]:
# got it means that it is in the excel sheet already

# Q_a = float(Qa[index])   #got it
# Q_b = float(Qb[index])   # got it
# T_e = Te[index]          # got it
# t_a = float(time_col[index,0]) #seconds    got it
# t_b = float(time_col[index,1]) #seconds    got it
# t_c = float(1/freq[index]-t_a-t_b)        #will be calculated in place
# t_b = t_c+t_b                             # will be calcullated in place
# V_p_ta = float(Vipa[index]/t_a) #volts  #will be calculated in place
# V_p_tb = float(Vipb[index]/(t_b+t_c))  #volts   # will be calculated in place
# V_p_tc = 0.0                                   # will be assigned in place
# Flow = flowrate[index]*6*(10**-5)-0.0001 #convert from % to m^3/s, see exccel "flow_calibration..."
# E = float(E_abs[index])                       # got it
# ns = width_ns[index]                          # got it
# temp_C_gas = temp_C[index]                    # will be set.
# frequency = freq[index]                       # got it
# volt = voltage[index]                         #got it
# sign = signature[index]                       # got it

In [6]:
from PyDBDdim.utils import DatasetProcess
DBD = DatasetProcess(r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_2cm_23mm_packing\Mesbah-DBD")
signature_list = DBD.folder_col
time_col, _= DBD.current_segment()
time_col = np.array(time_col)
t_index = np.array(_)
Vipab, Qab = DBD.get_two_region_VI()
Pabs_kj = DBD.int_power #this is actually worthless but just being complete.
E_abs = DBD.get_plasma_E() #joules?
#print(Vipab)
#print(Qab)
#print(time_col)
#print(_)
#print(signature_list)

Vipa = np.reshape(Vipab[:,0],(Vipab.shape[0],1))
Vipb = np.reshape(Vipab[:,1],(Vipab.shape[0],1))
Qa = np.reshape(Qab[:,0],(Qab.shape[0],1))
Qb = np.reshape(Qab[:,1],(Qab.shape[0],1))
t_a = np.reshape(time_col[:,0],(Qab.shape[0],1))
t_b = np.reshape(time_col[:,1],(Qab.shape[0],1))
Pabs_kj = np.reshape(Pabs_kj, (Pabs_kj.shape[0],1))
E_abs = np.reshape(E_abs, (E_abs.shape[0],1))
print(Pabs_kj.shape)
print(t_a.shape)
print(t_b.shape)
print(Qa.shape)
print(Qb.shape)
print(Vipa.shape)
print(Vipb.shape)
print(E_abs.shape)



matrix = np.hstack((Vipa, Vipb, Qa, Qb, Pabs_kj, E_abs, t_a, t_b))
variables = ['Vipa_Vs', 'Vipb_Vs', 'Qa_As', 'Qb_As', 'Pabs_kJ','E_abs','ta_s', 'tb_s']

(181, 1)
(181, 1)
(181, 1)
(181, 1)
(181, 1)
(181, 1)
(181, 1)
(181, 1)


In [7]:
print(matrix.shape)
print(Qa.shape)
print(Vipa.shape)

(181, 8)
(181, 1)
(181, 1)


In [8]:
print(signature_list)

['OES_20230505174100', 'OES_20230505180337', 'OES_20230505180802', 'OES_20230505181254', 'OES_20230505181648', 'OES_20230505182104', 'OES_20230505182511', 'OES_20230505182917', 'OES_20230505183348', 'OES_20230505184007', 'OES_20230505184410', 'OES_20230505184804', 'OES_20230505185222', 'OES_20230505185720', 'OES_20230505190138', 'OES_20230505190606', 'OES_20230505191039', 'OES_20230505191446', 'OES_20230505192036', 'OES_20230505192450', 'OES_20230505192854', 'OES_20230505193250', 'OES_20230505193705', 'OES_20230505194056', 'OES_20230505194515', 'OES_20230505194928', 'OES_20230505195257', 'OES_20230506134932', 'OES_20230506135343', 'OES_20230506135758', 'OES_20230506140224', 'OES_20230506140705', 'OES_20230506141148', 'OES_20230506151930', 'OES_20230506152439', 'OES_20230506152820', 'OES_20230506164248', 'OES_20230506164704', 'OES_20230506165039', 'OES_20230506165410', 'OES_20230506165731', 'OES_20230506170101', 'OES_20230506170538', 'OES_20230506171136', 'OES_20230506171552', 'OES_2023

In [10]:
# Put this in three parts:
#1) generate data frame from data above
#2) generate data frame from existing excel with data inputs and Temperature
#3) delete redundant, empty columns in old excel
#4) pd.merg the data frames
#5) load into a new excel


#1 df
df0 = pd.DataFrame(data=matrix,columns=[variables[i] for i in range(0,len(variables))]) #first sheet, list dimension vector
df0['signature_dbd'] = signature_list
#df1 = pd.DataFrame(data=signature_list,columns=['signature']) #second sheet, list nullspace vectors and associated variables



# df from existing excel

file_path = r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_2cm_23mm_packing\data_col_ks_original.xlsx"
df1 = pd.read_excel(file_path)  #, sheet_name = r"3cm_23mm_good"
df1 = df1.drop(columns = variables, errors = 'ignore')

#merge them
merged_df = pd.merge(df1, df0, left_on="signature", right_on="signature_dbd", how="inner")

merged_df.to_excel(
    r"C:\Users\vvmil\Documents\Python_Vmil\Jupyter_Notebooks\Plasma_He_calcs_cleanup\data\data_packing\data_2cm_23mm_packing\data_col_check_dbd_recheck_5_28.xlsx", 
    sheet_name = r"2cm_23mm",
    index=False)

In [11]:
df1

,flowrate_percent,width_ns,freq_kHz,voltage_kV,temp_C,power_w,706_667_587_K,706_587_K,706_728_K,706_728_587_K,signature,RF_direct_best,mis,height,packing_size
0,100,100,1,10,35.8,0.004843,1161,1184,1453,1147,OES_20230505174100,1404.28,48.72,2,2.3
1,100,120,1,10,36.2,0.009852,1291,1214,1436,1224,OES_20230505180337,1411.24,24.76,2,2.3
2,100,140,1,10,36.2,0.010229,1274,1189,1331,1195,OES_20230505180802,1479.87,-148.87,2,2.3
3,100,160,1,10,NaN,0.010072,1242,1183,1442,1194,OES_20230505181254,1431.71,10.29,2,2.3
4,100,180,1,10,36.6,0.007840,1280,1152,1420,1163,OES_20230505181648,1440.31,-20.31,2,2.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,30,100,6,12,NaN,0.046633,1164,1312,1661,1326,OES_20230508160705,1645.94,15.06,2,2.3
177,30,100,9,12,34.7,0.072041,1337,1482,2138,1506,OES_20230508161219,2032.20,105.80,2,2.3
178,30,100,1,14,35.6,0.009789,1246,1238,1405,1246,OES_20230508161635,1565.81,-160.81,2,2.3
179,30,150,1,14,35.2,0.011692,1175,1362,1442,1366,OES_20230508161956,1480.39,-38.39,2,2.3
